# Tutorial 3 - A Feedforward Spiking Neural Network

## Learn hot to
- simplify the LIF neuron to make it deep learning-friendly
- implement a feedforward SNN

> 실제 딥러닝에 적용하기 위해 LIF 뉴런 모델을 간소화하는 것과, 순전파 SNN을 구현해본다.

---